# 第9课：策略梯度方法（Policy Gradient Methods）

> value-based to policy-based

---

## Part 1：该方法的基本思路（Core Idea of Policy Gradient）

---

## 1. 动机：为什么要直接优化策略？

前几节课中（DP、TD、Q-learning 等方法）我们主要通过**估计值函数**（\$V(s)\$ 或 \$Q(s,a)\$）间接找到最优策略，如：

* 使用 **策略迭代（Policy Iteration）**；
* 用 **值函数更新指导行为选择（如 Q-learning）**。

但这些方法有以下 **局限性**：

| 局限         | 说明                                          |
| ---------- | ------------------------------------------- |
| ❌ 动作空间过大   | 动作空间一旦变成连续空间，无法枚举 max Q(s,a)，Q-learning 失效。 |
| ❌ 策略非显式表示  | 值函数方法只能间接推导策略，难以精确控制策略行为，不能直接优化策略结构。        |
| ❌ 难以表达概率策略 | 某些场景中（如博弈、探索问题）需要使用**随机策略**，而不是贪婪策略。        |

---

## 2. 核心思想：直接优化策略参数

我们定义一个**参数化的策略函数** \$\pi\_\theta(a|s)\$，表示在状态 \$s\$ 下采取动作 \$a\$ 的概率，由参数 \$\theta\$ 控制。

目标是：

> **直接调整 \$\theta\$ 以最大化某种“期望回报”目标函数 \$J(\theta)\$。**

即：

$$
\theta^* = \arg\max_\theta J(\theta)
$$

这类方法统称为：

> **策略梯度方法（Policy Gradient Methods）**

---

## 3. 策略函数形式举例

策略函数 \$\pi\_\theta(a|s)\$ 可以是以下任意形式：

| 类型   | 示例表达                                      | 应用场景               |
| ---- | ----------------------------------------- | ------------------ |
| 分类器型 | softmax(\$\theta^\top \phi(s,a)\$)        | 离散动作空间（e.g. Atari） |
| 高斯策略 | \$\mathcal{N}(\mu\_\theta(s), \sigma^2)\$ | 连续动作空间（e.g. 机械臂控制） |

---

## 4. 策略优化的目标函数

目标函数 \$J(\theta)\$ 一般设计为期望回报（不同形式稍有区别）：

* 在 episode 级别：

  $$
  J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ R(\tau) \right]
  $$

  其中 \$\tau\$ 是轨迹 trajectory，\$R(\tau)\$ 是整条轨迹回报。

* 或在状态级别：

  $$
  J(\theta) = \mathbb{E}_{s \sim d^\pi, a \sim \pi_\theta} \left[ Q^\pi(s,a) \right]
  $$

> **目标是最大化轨迹或状态下的期望回报**。

---

## 5. 策略梯度的更新法则（梯度上升）

使用 **梯度上升法** 对目标函数 \$J(\theta)\$ 进行优化：

$$
\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
$$

* \$\alpha\$ 是学习率；
* \$\nabla\_\theta J(\theta)\$ 是目标函数对策略参数的梯度；
* 如何计算这个梯度？——在后续部分会通过\*\*Policy Gradient Theorem（策略梯度定理）\*\*推导。

---

## 6. 与值函数方法对比

| 角度   | 值函数方法（如 Q-learning）    | 策略梯度方法                 |            |
| ---- | ---------------------- | ---------------------- | ---------- |
| 控制方式 | 通过 \$Q(s,a)\$ 决策（间接控制） | 显式建模策略 \$\pi\_\theta(a | s)\$（直接控制） |
| 动作空间 | 仅适用于离散动作               | 可用于**连续动作**控制          |            |
| 策略类型 | 贪婪策略、多数为确定性            | 原生支持随机策略               |            |
| 学习方式 | 目标值 + TD 误差            | 最大化回报目标，梯度上升           |            |
| 可解释性 | 难以解释策略行为               | 策略显式建模，可解释性强           |            |

---

## 7. 策略梯度方法的优缺点总结

### 优点：

* 可用于 **连续动作空间**；
* 可学习 **随机策略**；
* **可直接控制策略行为**，适用于策略结构复杂任务；
* 与 Actor-Critic、PPO、TRPO 等高级策略优化算法兼容。

### 缺点：

* 估计的梯度存在**高方差**，需要技巧（如 baseline）来减小；
* 收敛慢于基于值函数的方法；
* 需要更多样本数据（sample-inefficient）；

---

## 小结

> 策略梯度方法跳过了值函数建模的“间接步骤”，**直接对策略函数建模并优化其参数**，适用于复杂、连续、随机控制的问题。是现代强化学习中处理复杂策略控制问题的核心思路之一。







## Part 2：策略梯度方法的目标函数

在策略梯度方法中，我们通过最大化某种形式的 **期望回报目标函数 $J(\theta)$** 来优化参数化策略 $\pi_\theta(a|s)$。主要有两种目标定义：

---

### 1. Average Value Objective（起点状态）

假设从固定初始状态 $s_0$ 出发，目标是最大化该状态下的期望总回报：

$$
J(\theta) = V^{\pi_\theta}(s_0)
= \mathbb{E}_{\pi_\theta} \left[ \sum_{t=0}^{\infty} \gamma^t r_{t+1} \,\middle|\, s_0 \right]
$$

适用于 episodic task（从起点开始，如游戏第一关）。

---

### 2. Average Reward Objective（稳态平均奖励）

在 continuing task 中，假设系统达到稳态分布 $d^\pi(s)$，目标定义为长期平均奖励：

$$
J(\theta) = \lim_{T \to \infty} \frac{1}{T} \mathbb{E}_{\pi_\theta} \left[ \sum_{t=0}^{T-1} r_{t+1} \right]
= \mathbb{E}_{s \sim d^\pi, a \sim \pi_\theta} \left[ r(s,a) \right]
$$

适用于持续运行的任务，如推荐系统、通信网络等。

---

### 3. 统一表示形式

我们可以统一写作期望动作值的形式：

$$
J(\theta) = \sum_s d^\pi(s) \sum_a \pi_\theta(a|s) Q^\pi(s, a)
$$

其中：

* $d^\pi(s)$：状态在策略 $\pi$ 下的分布
* $Q^\pi(s,a)$：策略的动作值函数

---

### 4. 策略梯度定理（Policy Gradient Theorem）

我们希望最大化 $J(\theta)$，即计算其梯度：

$$
\nabla_\theta J(\theta) = \nabla_\theta \sum_s d^\pi(s) \sum_a \pi_\theta(a|s) Q^\pi(s,a)
$$

由于 $d^\pi(s)$ 也依赖 $\theta$，求导复杂。**策略梯度定理**给出一个简化公式：

$$
\boxed{
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim d^\pi, a \sim \pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a|s) Q^\pi(s,a) \right]
}
$$

---

### 5. 用 Monte Carlo 估计 $Q^\pi(s,a)$

![](./img/9_1.png)

由于真实的 $Q^\pi(s,a)$ 通常不可得，我们使用采样轨迹的回报 $G_t$ 替代：

$$
\nabla_\theta J(\theta) \approx \nabla_\theta \log \pi_\theta(a_t|s_t) \cdot G_t
$$

这就是 **REINFORCE 算法** 的基本形式（将在下一节介绍）。

---

### 6. 减少方差：引入 Baseline（优势函数）

为了减少估计的方差，我们引入基准函数 $b(s)$，保持无偏：

$$
\nabla_\theta J(\theta) = \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a|s) \cdot (Q^\pi(s,a) - b(s)) \right]
$$

若取 $b(s) = V^\pi(s)$，得到优势函数：

$$
A^\pi(s,a) = Q^\pi(s,a) - V^\pi(s)
$$

---

### 小结表格（公式已用 LaTeX 兼容格式）

| 目标函数类型         | 表达式                                                                | 应用场景               |       |
| -------------- | ------------------------------------------------------------------ | ------------------ | ----- |
| 起点状态值（s₀）      | $J(\theta) = V^{\pi_\theta}(s_0)$                                  | Episodic 任务        |       |
| 稳态平均奖励         | $J(\theta) = \mathbb{E}_{s,a}[r(s,a)]$                             | Continuing 任务      |       |
| 策略梯度公式         | ( \nabla\_\theta J(\theta) = \mathbb{E}\[\nabla\_\theta \log \pi(a | s) Q^\pi(s,a)] )   | 统一表达  |
| REINFORCE 梯度估计 | ( \nabla\_\theta J(\theta) \approx \nabla\_\theta \log \pi(a\_t    | s\_t) \cdot G\_t ) | 无模型估计 |
| 引入 baseline    | 使用 $Q^\pi(s,a) - b(s)$，常设 $b(s) = V^\pi(s)$                        | 降低方差，提高稳定性         |       |






## Part 3：梯度上升算法和 REINFORCE

---

### 1 策略梯度的基本优化思路

策略梯度方法的核心思想是：**直接优化策略函数** \$\pi\_\theta(a|s)\$，使期望回报 \$J(\theta)\$ 最大化。

由于 \$J(\theta)\$ 是可微的，我们可以使用梯度上升法来迭代优化：

$$
\boxed{
\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
}
$$

其中：

* \$\alpha\$：学习率
* \$\nabla\_\theta J(\theta)\$：目标函数的梯度，由策略梯度定理给出

---

### 2 REINFORCE 算法（Williams, 1992）

REINFORCE 是一种**蒙特卡洛策略梯度方法**，用于在**不依赖环境模型**的情况下估计梯度。

---

#### 2.1 梯度估计公式

由策略梯度定理（忽略状态分布的梯度项）：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t \right]
$$

其中：

* \$G\_t = \sum\_{k=0}^{\infty} \gamma^k r\_{t+k+1}\$：从时间步 \$t\$ 起的折扣回报
* 梯度是对策略的参数 \$\theta\$ 进行的

---

#### 2.2 算法步骤（一个 Episode）

假设我们以 episode 为单位更新：

1. 初始化参数 \$\theta\$
2. 对每个 episode：

   * 使用当前策略 \$\pi\_\theta\$ 与环境交互，收集轨迹：
     \$(s\_0, a\_0, r\_1, s\_1, a\_1, r\_2, \dots, s\_T)\$
   * 计算每个时间步 \$t\$ 的回报 \$G\_t\$
   * 对每个 \$t\$ 执行梯度更新：

$$
\theta \leftarrow \theta + \alpha \cdot \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t
$$

---

#### 2.3 伪代码（完整）

```python
initialize policy parameters θ
repeat for each episode:
    generate an episode: s0, a0, r1, s1, ..., sT
    for each step t in the episode:
        Gt ← discounted return from t
        θ ← θ + α ∇θ log πθ(at | st) · Gt
```

---

### 3 数学推导简述

我们从期望形式出发：

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ R(\tau) \right] = \sum_{\tau} P(\tau; \theta) \cdot R(\tau)
$$

对参数 \$\theta\$ 求导：

$$
\nabla_\theta J(\theta) = \sum_\tau \nabla_\theta P(\tau; \theta) \cdot R(\tau)
= \sum_\tau P(\tau; \theta) \cdot \nabla_\theta \log P(\tau; \theta) \cdot R(\tau)
$$

因为轨迹概率 \$P(\tau; \theta)\$ 是多个动作概率的乘积：

$$
\log P(\tau; \theta) = \sum_{t=0}^{T-1} \log \pi_\theta(a_t | s_t)
$$

所以：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \sum_t \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t \right]
$$

---

### 4 引入 Baseline（降低方差）

我们可以使用 baseline（基准值）来减少方差，而不会引入偏差：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot (G_t - b(s_t)) \right]
$$

常见选择：\$b(s\_t) = V^\pi(s\_t)\$，此时 \$G\_t - V^\pi(s\_t)\$ 就是优势函数 \$A^\pi(s\_t, a\_t)\$。

---

### 5 特点总结

| 优点                  | 缺点              |
| ------------------- | --------------- |
| 不依赖环境模型（Model-free） | 方差较大            |
| 可应用于离散/连续动作空间       | 收敛慢，需完整 episode |
| 理论清晰，易于理解与实现        | 对奖励稀疏、长回报问题不友好  |

---

### 6 小结（公式对比）

| 表达式类型       | 数学公式                                                                                     |
|------------------|----------------------------------------------------------------------------------------------|
| 策略梯度定理     | $\nabla_\theta J(\theta) = \mathbb{E}_\pi\left[ \nabla_\theta \log \pi_\theta(a|s) Q^\pi(s,a) \right]$ |
| REINFORCE        | $\nabla_\theta J(\theta) \approx \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot G_t$               |
| 引入 Baseline    | $\nabla_\theta J(\theta) \approx \nabla_\theta \log \pi_\theta(a_t | s_t) \cdot (G_t - b(s_t))$     |



---

## 本课总结

* 策略梯度方法通过参数化策略直接优化目标函数
* 策略梯度定理提供了梯度表达式，方便计算
* REINFORCE 算法用蒙特卡洛方法估计梯度
* 方差控制和样本效率是进一步研究方向

# Q1 策略梯度定理（Policy Gradient Theorem）推导

我们希望最大化：

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ R(\tau) \right]
$$

使用梯度上升法更新参数：

$$
\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)
$$

---

## 转换为状态-动作分布下的形式：

$$
J(\theta) = \sum_s d^\pi(s) \sum_a \pi_\theta(a|s) Q^\pi(s,a)
$$

求导得到：

$$
\nabla_\theta J(\theta) = \sum_s d^\pi(s) \sum_a \nabla_\theta \pi_\theta(a|s) Q^\pi(s,a)
$$

引入 log-derivative trick：

$$
\nabla_\theta \pi_\theta(a|s) = \pi_\theta(a|s) \nabla_\theta \log \pi_\theta(a|s)
$$

最终推导出策略梯度定理：

$$
\boxed{
\nabla_\theta J(\theta) = \mathbb{E}_{s \sim d^\pi, a \sim \pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a|s) Q^\pi(s,a) \right]
}
$$

---

## 使用 $G_t$ 估计 $Q^\pi(s,a)$：

$$
\nabla_\theta J(\theta) \approx \nabla_\theta \log \pi_\theta(a_t|s_t) \cdot G_t
$$

引入 baseline 后的形式为：

$$
\nabla_\theta J(\theta) = \mathbb{E}_{s,a} \left[ \nabla_\theta \log \pi_\theta(a|s) (Q^\pi(s,a) - b(s)) \right]
$$


